In [565]:
import polars as pl

In [745]:
cahps_df = pl.read_parquet('../../003_data/002_clean-data/national_cms_dialysis-facility_cahps-data.parquet')
facility_df = pl.read_parquet('../../003_data/002_clean-data/national_cms_dialysis-facility_data.parquet')
ballot_measures_df = pl.read_parquet('../../003_data/001_raw-data/2018-2022_ballot-measure_sub-county_data.parquet')
supp_facility_df = pl.read_parquet('../../003_data/001_raw-data/2013-2023_CHHS_dialysis-facility_data.parquet')


In [746]:
# Merging the cahps and facility data

merged_cms_df = facility_df.join(
    cahps_df,
    on=['provider_number', 'year'],
    how='left'
)

# Dropping the right-side columns, which are duplicates
merged_cms_df = merged_cms_df.drop(merged_cms_df.select(pl.col("^.*_right$")).columns)


In [747]:
# Casting year columns to string to match the format in the merged cms dataframe

supp_facility_df = supp_facility_df.with_columns(pl.col("year").cast(pl.Utf8))

ballot_measures_df = ballot_measures_df.with_columns(pl.col("year").cast(pl.Utf8))


In [748]:
ballot_measures_df

year,county,sub_county,district_id,geo_type,vote_type,vote_count
str,str,str,str,str,str,str
"""2022""","""Alameda""","""County Supervisorial 1""","""1""","""county_supervisorial_district""","""yes""","""31161"""
"""2022""","""Alameda""","""County Supervisorial 2""","""2""","""county_supervisorial_district""","""yes""","""28778"""
"""2022""","""Alameda""","""County Supervisorial 3""","""3""","""county_supervisorial_district""","""yes""","""34338"""
"""2022""","""Alameda""","""County Supervisorial 4""","""4""","""county_supervisorial_district""","""yes""","""33897"""
"""2022""","""Alameda""","""County Supervisorial 5""","""5""","""county_supervisorial_district""","""yes""","""54523"""
…,…,…,…,…,…,…
"""2018""","""Yuba""","""State Assembly District 3""","""3""","""state_assembly_district""","""no""","""12845"""
"""2018""","""Yuba""","""Board of Equalization District…","""1""","""state_board_of_equalization_di…","""no""","""12845"""
"""2018""","""Yuba""","""Marysville""",null,"""city""","""no""","""1857"""


In [749]:
# In this cell we're lowercasing all county and city names in the CMS dataframes
# This will allow us to join the combined dataframe with the supplemental dataframe using county and city
# It also serves as an extra data validation step. At this point, there shouldn't be any city or county names that are not 

# Lowercasing all county and city names in the CMS dataframes

facility_df = (
    facility_df
    .with_columns(
        pl.col("county")
        .str.to_lowercase()
        .alias("county"),
        pl.col("city")
        .str.to_lowercase()
        .alias("city")
    )
)

cahps_df = (
    cahps_df
    .with_columns(
        pl.col("county")
        .str.to_lowercase()
        .alias("county"),
        pl.col("city")
        .str.to_lowercase()
        .alias("city")
    )
)

merged_cms_df = (
    merged_cms_df
    .with_columns(
        pl.col("county")
        .str.to_lowercase()
        .alias("county"),
        pl.col("city")
        .str.to_lowercase()
        .alias("city")
    )
)

# Lowercasing all county and sub-county names in the SOS ballot measures dataframe

ballot_measures_df = (
    ballot_measures_df
    .with_columns(
        pl.col("county")
        .str.to_lowercase()
        .alias("county"),
        pl.col("sub_county")
        .str.to_lowercase()
        .alias("sub_county")
    )
)

In [750]:
# Here we take an extra data validation step: checking for unique counties in the datasets
# At this point, there shouldn't be any county names that are unique to any dataframe


# Getting unique counties from all dataframes
ballot_counties = set(ballot_measures_df['county'].unique())
facility_counties = set(facility_df['county'].unique())
cahps_counties = set(cahps_df['county'].unique())

print("Unique counties in ballot_measures_df:")
print(sorted(ballot_counties))

print("\nUnique counties in facility_df:")
print(sorted(facility_counties))

print("\nUnique counties in cahps_df:")
print(sorted(cahps_counties))

# Finding counties that are in one dataframe but not others
counties_only_in_ballot = ballot_counties - facility_counties - cahps_counties
counties_only_in_facility = facility_counties - ballot_counties - cahps_counties
counties_only_in_cahps = cahps_counties - ballot_counties - facility_counties

print("\nCounties only in ballot_measures_df:")
print(sorted(counties_only_in_ballot))

print("\nCounties only in facility_df:")
print(sorted(counties_only_in_facility))

print("\nCounties only in cahps_df:")
print(sorted(counties_only_in_cahps))

# Checking for potential mismatches due to formatting
all_counties = ballot_counties.union(facility_counties).union(cahps_counties)
potential_mismatches = [county for county in all_counties if any(
    county != other_county and county in other_county
    for other_county in all_counties
)]

print("\nPotential county name mismatches:")
print(sorted(potential_mismatches))

Unique counties in ballot_measures_df:
['alameda', 'alpine', 'amador', 'butte', 'calaveras', 'colusa', 'contra costa', 'del norte', 'el dorado', 'fresno', 'glenn', 'humboldt', 'imperial', 'inyo', 'kern', 'kings', 'lake', 'lassen', 'los angeles', 'madera', 'marin', 'mariposa', 'mendocino', 'merced', 'modoc', 'mono', 'monterey', 'napa', 'nevada', 'orange', 'placer', 'plumas', 'riverside', 'sacramento', 'san benito', 'san bernardino', 'san diego', 'san francisco', 'san joaquin', 'san luis obispo', 'san mateo', 'santa barbara', 'santa clara', 'santa cruz', 'shasta', 'sierra', 'siskiyou', 'solano', 'sonoma', 'stanislaus', 'sutter', 'tehama', 'trinity', 'tulare', 'tuolumne', 'ventura', 'yolo', 'yuba']

Unique counties in facility_df:
['', 'alameda', 'butte', 'columbia', 'colusa', 'contra costa', 'del norte', 'el dorado', 'fresno', 'humboldt', 'imperial', 'inyo', 'kern', 'kings', 'lake', 'los angeles', 'madera', 'marin', 'mendocino', 'merced', 'monterey', 'napa', 'nevada', 'orange', 'placer',

In [751]:
# Filtering the supplemental facility information dataframe to include on dialysis clinics

supp_facility_df = supp_facility_df.filter(pl.col("LIC_CAT") == "Chronic Dialysis Clinic")

In [752]:
# Selecting facility ids and geographic features to merge with CMS data
# Stripping extraneous numbers from facility ids to match the id format in CMS data
# Renaming FAC_NO provider_number to match the id format in CMS data

supp_facility_df = (supp_facility_df
 .with_columns(
     pl.col("FAC_NO")
     .alias("provider_number"))
 .with_columns(
     pl.col("COUNTY")
     .alias("county"))
 .with_columns(
     pl.col("FAC_NAME")
     .alias("facility_name"))
 .with_columns(
     pl.col("FAC_ZIP")
     .alias("zip_code"))
 .select([
     "year",
    "provider_number",
    "facility_name",
    "county",
    "zip_code",
    "ASSEMBLY_DIST",
    "SENATE_DIST",
    "CONGRESS_DIST",
    "CENS_TRACT",
    "LONGITUDE",
    "LATITUDE"
    ])
)

In [753]:
# Merging datasets using standardized facility names

def standardize_facility(df):
    df = df.with_columns(
        pl.col('facility_name')
        .str.to_lowercase()
        .str.strip_chars()
        .str.replace(r'[^a-z0-9]', '')  # Remove all non-alphanumeric characters
        .alias('std_facility_name')
    )
    
    return df

supp_facility_df = standardize_facility(supp_facility_df)
cahps_df = standardize_facility(cahps_df)
facility_df = standardize_facility(facility_df)
merged_cms_df = standardize_facility(merged_cms_df)

# Here we're using multiple columns for matching
# This is important because some facilities are essentially chains, so they have the same name regardless of zip code
# We also tried using a fuzzy merge to match the facility names, but decided that having an accurate merge was more important than using a more complicated, but  matching algorithm


supplemented_cms_df = merged_cms_df.join(
    supp_facility_df,
    on=['std_facility_name', 'zip_code'],
    how='left'
)


In [755]:
# The three districts with the fewest dialysis facilities are 63, 23, and 67
# 63 is in a mountainous part of sacramento, 67 is in a mountainous part of santa clara, and 23 is a land area occupied mostly by diseneyland

# The three districts with the most dialysis facilities are 66, 57, and 48
# All are in the LA area

supplemented_cms_df.filter(pl.col("ASSEMBLY_DIST").is_not_null()).group_by("ASSEMBLY_DIST").len().sort("len", descending=True)


ASSEMBLY_DIST,len
f64,u32
66.0,220
57.0,188
48.0,156
3.0,120
53.0,114
…,…
15.0,9
13.0,9
67.0,5


In [756]:
# Reshaping the ballot measures dataframe to merge with CMS cahps and facility data
# We can merge city-level votes directly with the original cahps and facility data

ballot_measures_df_city = (ballot_measures_df.filter(pl.col("geo_type") == "city")
 .pivot(
    index=['county', 'year', 'sub_county', 'vote_type'],
    on='geo_type',
    values='vote_count'
).with_columns(
    pl.col("city")
    .cast(pl.Int64)
    .alias("vote_count"),
    pl.col("sub_county")
    .alias("city")
)
.drop("sub_county")
)

# Reshaping the ballot measures dataframe to merge with CMS cahps and facility data
# We can merge Assembly-District-level votes with the cahps and facility data after
# they've been merged with the supplemental California Health and Human Services data

ballot_measures_df_assembly_district = (ballot_measures_df.filter(pl.col("geo_type") == "state_assembly_district")
 .pivot(
    index=['year', 'county', 'district_id', 'vote_type'],
    on='geo_type',
    values='vote_count')
 .with_columns(
    pl.col("state_assembly_district")
    .cast(pl.Int64)
    .alias("vote_count"),
    pl.col("district_id")
    .cast(pl.Float64)
    .alias("ASSEMBLY_DIST")
)
.drop("state_assembly_district")
)

In [766]:
# Creating a list of variables to select from the cahps dataframe
# These will be used for analysis

cms_variables_of_interest_by_city = [
    "year",
    "county",
    "city",
    "provider_number",
    "profit_or_nonprofit",
    "chain_organization",
    "ichcahps_data_availability_code",
    "five_star_data_availability_code",
    "linearized_score_of_rating_of_the_dialysis_facility",
    "five_star",
    "ich_cahps_survey_of_patients_experiences_star_rating",
    "linearized_score_of_rating_of_the_dialysis_center_staff",
    "patient_transplant_waitlist_data_availability_code",
    "standardized_first_kidney_transplant_waitlist_ratio", #higher is better
    "_of_dialysis_stations",
    "patient_hospital_readmission_category",
    "mortality_rate_facility"
]

# Merging the cahps dataframe with city-level votes
# Only selecting variables of interest and filtering for facilitites with available data
# Unsure what to do with year
# Including all years in the dataset increases the number of cities with available data from 242 to 255

merged_cms_df_with_votes_by_city = (
    merged_cms_df
    .select(pl.col(cms_variables_of_interest_by_city))
    # .filter(pl.col("year").is_in(pl.Series(["2018", "2020", "2022"])))
    .filter(pl.col("ichcahps_data_availability_code") == "1")
    .join(
        ballot_measures_df_city,
        on=['year','county', 'city'],
        how='left'
    )
)

In [767]:
merged_cms_df_with_votes_by_city.select("city").unique().count()

city
u32
255


In [759]:
# Creating a list of variables to select from the cahps dataframe
# These will be used for analysis

cms_variables_of_interest_by_assembly_district = [
    "year",
    "county",
    "city",
    "provider_number",
    "ASSEMBLY_DIST",
    "CENS_TRACT",
    "LONGITUDE",
    "LATITUDE",
    "profit_or_nonprofit",
    "chain_organization",
    "ichcahps_data_availability_code",
    "five_star_data_availability_code",
    "linearized_score_of_rating_of_the_dialysis_facility",
    "five_star",
    "ich_cahps_survey_of_patients_experiences_star_rating",
    "linearized_score_of_rating_of_the_dialysis_center_staff",
    "patient_transplant_waitlist_data_availability_code",
    "standardized_first_kidney_transplant_waitlist_ratio", #higher is better
    "_of_dialysis_stations",
    "patient_hospital_readmission_category",
    "mortality_rate_facility"
]

# Merging the cahps dataframe with Assembly-District-level votes
# Only selecting variables of interest and filtering for facilities with known assembly districts and available data
# Unsure what to do with year
# Including all years in the dataset increases the number of assembly districts with available data from 20 to 76

supplemented_cms_df_with_votes_by_assembly_district = (
    supplemented_cms_df
    .select(pl.col(cms_variables_of_interest_by_assembly_district))
    .filter(pl.col("ASSEMBLY_DIST").is_not_null())
    # .filter(pl.col("year").is_in(pl.Series(["2018", "2020", "2022"])))
    .filter(pl.col("ichcahps_data_availability_code") == "1")
    .join(
        ballot_measures_df_assembly_district,
        on=['year','county', 'ASSEMBLY_DIST'],
        how='left'
    )
)

In [760]:
supplemented_cms_df_with_votes_by_assembly_district.select("ASSEMBLY_DIST").unique().count()


ASSEMBLY_DIST
u32
76
